Sometimes we need to print something< for exmple arguments for debugging

In [18]:
def max(*args):
    """finds the largest argument."""
    print(f"max{args}=...")
    ret=0
    for x in args:
        ret=ret if x<ret else x
    print(f"max{args}={ret}")
    return ret
def foo():
    max(-10,-1,-3)
foo()


max(-10, -1, -3)=...
max(-10, -1, -3)=0
max


we can define high level function, that implements the same mechanism for each function

In [19]:
def trace(f):
    def inner(*args, **kwargs):
        call =",".join(
        [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
        )
        print(f"{f.__name__}({call})=...")
        ret = f(*args,**kwargs)
        print(f"{f.__name__}({call})={ret}")
        return ret
    return inner

def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret

max=trace(max)
def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max(-10,-1,-3)=...
max(-10,-1,-3)=0
inner


It's possible not to overwrite function implicitly. We can just use decorator @trace. But we still need to implement trace function!
Also it's a good idea to add DEBUG flag, which allow us to choose behavior

In [20]:
DEBUG = True

def trace(f):
    if not DEBUG:
        return f
    def inner(*args, **kwargs):
        call =",".join(
        [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
        )
        print(f"{f.__name__}({call})=...")
        ret = f(*args,**kwargs)
        print(f"{f.__name__}({call})={ret}")
        return ret
    inner.__name__=f.__name__
    inner.__doc__=f.__doc__
    inner.__module=f.__module__
    return inner

@trace
def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret


def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max(-10,-1,-3)=...
max(-10,-1,-3)=0
max


let's use another function for function attributes

In [31]:
def update_wrapper(wrapped,wrapper):
    for attr in["__name__","__doc__","__module__"]:
        setattr(wrapper,attr,getattr(wrapped,attr))
    wrapper.__wrapped__=wrapped
def trace(f):
    if not DEBUG:
        return f
    def inner(*args, **kwargs):
        call =",".join(
        [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
        )
        print(f"{f.__name__}({call})=...")
        ret = f(*args,**kwargs)
        print(f"{f.__name__}({call})={ret}")
        return ret
    update_wrapper(f,inner)
    return inner
@trace
def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret
def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max(-10,-1,-3)=...
max(-10,-1,-3)=0
max


Now we use it as a decorator by using partial call. We need partioal call to rid of first argument

In [30]:
import functools
DEBUG= True
def update_wrapper(wrapped,wrapper):
    for attr in["__name__","__doc__","__module__"]:
        setattr(wrapper,attr,getattr(wrapped,attr))
    return wrapper
def trace(f):
    if not DEBUG:
        return f
    wraps=functools.partial(update_wrapper,f)
    @wraps
    def inner(*args, **kwargs):
        call =",".join(
        [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
        )
        print(f"{f.__name__}({call})=...")
        ret = f(*args,**kwargs)
        print(f"{f.__name__}({call})={ret}")
        return ret
    update_wrapper(f,inner)
    return inner
@trace
def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret
def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max(-10,-1,-3)=...
max(-10,-1,-3)=0
max


In [35]:
#another way
DEBUG= True
def update_wrapper(wrapped,wrapper):
    for attr in["__name__","__doc__","__module__"]:
        setattr(wrapper,attr,getattr(wrapped,attr))
    return wrapper

def wraps(f):
    def deco(g):
        update_wrapper(f,g)
        return g
    return deco

def trace(f):
    if not DEBUG:
        return f
    @wraps(f)
    def inner(*args, **kwargs):
        call =",".join(
        [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
        )
        print(f"{f.__name__}({call})=...")
        ret = f(*args,**kwargs)
        print(f"{f.__name__}({call})={ret}")
        return ret
    update_wrapper(f,inner)
    return inner
@trace
def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret
def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max(-10,-1,-3)=...
max(-10,-1,-3)=0
max


Actually these decorators are implemented in functools

In [1]:
import functools
DEBUG= True

def trace(f):
    if not DEBUG:
        return f
    @functools.wraps(f)
    def inner(*args, **kwargs):
        call =",".join(
            [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
        )
        print(f"{f.__name__}({call})=...")
        ret = f(*args,**kwargs)
        print(f"{f.__name__}({call})={ret}")
        return ret
    return inner
@trace
def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret
def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max(-10,-1,-3)=...
max(-10,-1,-3)=0
max


Better practice is to write debug information to stderr(stream that is used in case of errors)

In [4]:
import functools
import sys

DEBUG= True

def trace(stream=sys.stdout):
    def decorator(f):
        if not DEBUG:
            return f
        @functools.wraps(f)
        def inner(*args, **kwargs):
            call =",".join(
                [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
            )
            print(f"{f.__name__}({call})=...", file=stream)
            ret = f(*args,**kwargs)
            print(f"{f.__name__}({call})={ret}",file =stream)
            return ret
        return inner
    return decorator

@trace(sys.stderr)
def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret
def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max


max(-10,-1,-3)=...
max(-10,-1,-3)=0


if we don't want use parameter every time, we can do this kind of magic: When we give to function stream, f is None and we return decorator trace agein with new stream.

In [11]:
import functools
import sys

DEBUG= True

def trace(f=None,*,stream=sys.stdout):
    if f is None:
        return functools.partial(trace, stream=stream)
    if not DEBUG:
        return f
    @functools.wraps(f)
    def inner(*args, **kwargs):
        call =",".join(
             [str(a) for a in args]+[f"{k}={v}" for k,v in kwargs]
        )
        print(f"{f.__name__}({call})=...", file=stream)
        ret = f(*args,**kwargs)
        print(f"{f.__name__}({call})={ret}",file =stream)
        return ret
    return inner

@trace(stream =sys.stderr)
def max(*args):
    """finds the largest argument."""
    ret=0
    for x in args:
        ret=ret if x<ret else x
    return ret
def foo():
    max(-10,-1,-3)
foo()
print(max.__name__)

max


max(-10,-1,-3)=...
max(-10,-1,-3)=0


How limit function to be called only once?

In [16]:
def once(f):
    called = False 
    def inner(*args,**kwargs):
        nonlocal called
        if not called:
            called=True
            res=f(*args,**kwargs)
            assert res is None
    return inner


@once
def init_logger():
    """Call this at most once"""
    print("initializing logger")


def foo():
    init_logger()
    
if __name__ == '__main__':
    foo()
    foo()

initializing logger


When we need mark function as deprecated.

In [22]:
import sys
import warnings
def deprecated(f):
    def inner(*args,**kwargs):
        warnings.warn(f"{f.__name__} is deprecated",
                     category=DeprecationWarning)
        print(f"Don't use {f.__name__}, use ... instead", file=sys.stderr)
        return f(*args,**kwargs)
    return inner


@deprecated
def init_logger():
    """Call this at most once"""
    print("initializing logger")


def foo():
    init_logger()
    
if __name__ == '__main__':
    foo()
    foo()

initializing logger
initializing logger


D:\Tools\anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: init_logger is deprecated
  
Don't use init_logger, use ... instead
Don't use init_logger, use ... instead


For profiling code we also can use decorators. Also we add memoize decorator to cache values

In [25]:
import time
import functools
def profile(f):
    @functools.wraps(f)
    def inner(*args,**kwargs):
        start = time.perf_counter()
        res=f(*args,**kwargs)
        elapsed=time.perf_counter()-start
        inner.__n_calls__+=1
        inner.__total_time__+=elapsed
        return res
    inner.__n_calls__=0
    inner.__total_time__=0
    return inner

def memoize(f):
    cache ={}
    @functools.wraps(f)
    def inner(*args,**kwargs):
        key=(args,frozenset(kwargs.items()))
        if key not in cache:
            cache[key]=f(*args,**kwargs)
        return cache[key]
    inner.__cache__=cache
    return inner

@profile #oder of decoratora is essential profile(memoize(profile(fib)))
@memoize #can use functools.lru_cache(maxsize=None)
def fib(n):
    return 1 if n<=1 else fib(n-1)+ fib(n-2)

if __name__== "__main__":
    print(fib(22))
    print(fib.__n_calls__)
    print(fib.__total_time__)

28657
43
0.0010570000022198656


Decorator singledispatch. It allow us to do different inplementations for function.

In [28]:
@functools.singledispatch
def json(x):
    assert False, f"json not supported for {type(x)}"

@json.register(type(None))
def _(x):
    return "null"
@json.register(int)
def _(x):
    return str(x)
@json.register(list)
def _(xs):
    contents = ",".join(json(x) for x in xs)
    return f"[{contents}]"

print(json(None))
print(json(92))
print(json([92,None]))
print(json("wefdfdgf"))

null
92
[92,null]


AssertionError: json not supported for <class 'str'>

In functools we have another function reduce, which works like left fold in ocaml

In [31]:
import functools
res=functools.reduce(lambda x,y:x+","+y, ["a","b","c"], "initial") 
print(res)
def f(x ,y):
    return x +"_"+y
res=functools.reduce(f, ["a","b","c"], "initial") #f(f(f("initial","a"),"b"),"c")
print(res)
print(f(f(f("initial","a"),"b"),"c"))

initial,a,b,c
initial_a_b_c
initial_a_b_c
